In [ ]:
#! pip install wget

In [ ]:

#import wget

In [ ]:

#url = "https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip"

In [ ]:

#wget.download(url,"")

In [ ]:

#! unzip data.zip

In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data_dir= 'data'

In [ ]:
track = pd.read_csv('data/driving_log.csv')

In [ ]:
track.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
traincsv , testcsv = train_test_split(track,test_size =0.05)

In [ ]:
testcsv.shape

(402, 7)

In [ ]:
def my_gen(testcsv,data_dir,Batch_size):
    while True:
        testcsv = testcsv.sample(frac = 1)
        end = 0
        iter_i = int(testcsv.shape[0]/Batch_size)
        for i in range(iter_i+1):
            start = end
            if(i==iter_i):
                end = None
            else:
                end = start+Batch_size
            
            rowselect = testcsv[start:end]
            img = []
            labels =[]
            for rows in rowselect.iterrows():
                center = cv2.cvtColor(cv2.imread(data_dir+"/"+"{}".format(rows[1]['center'].strip())),cv2.COLOR_BGR2RGB)
                left = cv2.cvtColor(cv2.imread(data_dir+"/"+"{}".format(rows[1]['left'].strip())),cv2.COLOR_BGR2RGB)
                right = cv2.cvtColor(cv2.imread(data_dir+"/"+"{}".format(rows[1]['right'].strip())),cv2.COLOR_BGR2RGB)
                steer = float(rows[1]['steering'])
                
                img.append(center)
                labels.append(steer)
                img.append(left)
                labels.append(steer+0.2)
                img.append(right)
                labels.append(steer-0.2)
                
                
                #Augumentation
                
                img.append(cv2.flip(center,1))
                labels.append(steer*-1)
                img.append(cv2.flip(left,1))
                labels.append((steer+0.2)*-1)
                img.append(cv2.flip(right,1))
                labels.append((steer-0.2)*-1)
                
                
            img = np.array(img)
            labels = np.array(labels)

            
            yield img,labels
            
        
 

In [ ]:
train_gen = my_gen(traincsv,data_dir,32)
test_gen = my_gen(testcsv,data_dir,32)


In [ ]:
from keras.models import Sequential,Model
from keras.layers import Cropping2D
from keras.layers.core import Dense , Activation ,Flatten , Lambda
from keras.layers.convolutional import Conv2D
import tensorflow as tf

In [ ]:
def behavioral_model():
    model = Sequential()
    model.add(Lambda(lambda x:(x/255)-0.5,input_shape =(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))
    model.add(Conv2D(24,(5,5),activation='relu'))
    model.add(Conv2D(36,(5,5),activation='relu'))
    model.add(Conv2D(48,(5,5),activation='relu'))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(Flatten())
    model.add(Dense(100,activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(1))
    return model

In [ ]:
model = behavioral_model()

In [ ]:
model.compile(optimizer='Adam',loss='mse')

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
model.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=traincsv.shape[0],validation_steps=testcsv.shape[0],epochs = 10,verbose=1,callbacks=my_callbacks)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
   1/7634 [..............................] - ETA: 0s - loss: 0.0355WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
7634/7634 [==============================] - 9119s 1s/step - loss: 0.0068 - val_loss: 0.0107
Epoch 2/10
7634/7634 [==============================] - 8976s 1s/step - loss: 4.8684e-04 - val_loss: 0.0108
Epoch 3/10
2989/7634 [==========>...................] - ETA: 1:28:40 - loss: 3.1070e-04